In [ ]:
%pip install -q kagglehub
import kagglehub

# Download latest version
path = kagglehub.dataset_download("salader/dogs-vs-cats")

print("Path to dataset files:", path)

Note: you may need to restart the kernel to use updated packages.
